#### Note

In [130]:
# pd.set_option('display.max_rows', 100) # 최대 display option set
# pd.reset_option('display.max_rows') # 최대 display option reset

#### IMPORT

In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### 08-19 OHCA 합치기

In [ ]:
# ohca_08 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_08.sas7bdat', encoding = 'euc-kr')
# ohca_09 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_09.sas7bdat', encoding = 'euc-kr')
# ohca_10 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_10.sas7bdat', encoding = 'euc-kr')
# ohca_11 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_11.sas7bdat', encoding = 'euc-kr')
# ohca_12 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_12.sas7bdat', encoding = 'euc-kr')
# ohca_13 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_13.sas7bdat', encoding = 'euc-kr')
# ohca_14 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_14.sas7bdat', encoding = 'euc-kr')
# ohca_16 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_16.sas7bdat', encoding = 'euc-kr')
# ohca_17 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_17.sas7bdat', encoding = 'euc-kr')
# ohca_18 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_18.sas7bdat', encoding = 'euc-kr')
# ohca_19 = pd.read_sas('data\\급성심장정지조사(08-19)SAS\\ohca_19.sas7bdat', encoding = 'euc-kr')

In [ ]:
# ohca_15 = pd.read_table('data\\급성심장정지조사(08-19)SAS\\ohca_15.txt', encoding = 'utf-8', sep='\t')

In [ ]:
# ohca_15

In [ ]:
# ohca_all = pd.concat([ohca_08, ohca_09, ohca_10, ohca_11, ohca_12, ohca_13, ohca_14, ohca_15, ohca_16, ohca_17, ohca_18, ohca_19], axis = 0)
# ohca_all.reset_index(drop = True, inplace = True)
# ohca_all

In [ ]:
# ohca_all.to_csv('data\\급성심장정지조사(08-19)SAS\\ohca_all.csv', encoding = 'euc-kr', index = False)

#### 08-19 합친 ohca_all 다시 불러오기

In [220]:
ohca_all = pd.read_csv('data\\급성심장정지조사(08-19)SAS\\ohca_all.csv', encoding = 'euc-kr', low_memory = False)

##### ADD_CITY, ADD_DIST 합치기

In [221]:
ohca_all['H_ADD_CITY_DIST'] = ohca_all['H_ADD_CITY'] + ' ' + ohca_all['H_ADD_DIST']
ohca_all['P_ADD_CITY_DIST'] = ohca_all['P_ADD_CITY'] + ' ' + ohca_all['P_ADD_DIST']

In [222]:
# NA check 

ohca_all.isna().sum()

YEAR                    0
H_ADMINCODE            48
H_ADD_CITY              0
H_ADD_DIST              0
P_ADMINCODE          8639
                    ...  
OP8_ROSC_DT        323707
OP8_ROSC_TM        323707
BYSTANDER_AED      324036
H_ADD_CITY_DIST         0
P_ADD_CITY_DIST         0
Length: 233, dtype: int64

In [223]:
ohca_all['P_ADMINCODE'].isna().sum()

8639

In [224]:
ohca_all['PRE_ER_ARREST_DT'].isna().sum()

187122

##### PRE_ER_ARREST_DT NA 처리

In [242]:
# str 형식으로 바꾸기

ohca_all['PRE_ER_ARREST_DT'] = ohca_all['PRE_ER_ARREST_DT'].astype(str)

In [243]:
len(ohca_all.loc[ohca_all['PRE_ER_ARREST_DT'].isna() == True])

0

In [244]:
# 'PRE_ER_ARREST_DT' 20080101.0 형식에서 .0 자르기

ohca_all['PRE_ER_ARREST_DT']= ohca_all['PRE_ER_ARREST_DT'].str[0:8]
ohca_all

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,...,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST,P_ADD_CITY_DIST
0,2008,11410.0,서울,서대문구,11680.0,서울,강남구,1,58,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서대문구,서울 강남구
1,2008,11320.0,서울,도봉구,11305.0,서울,강북구,2,47,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구,서울 강북구
2,2008,11110.0,서울,종로구,11440.0,서울,마포구,2,81,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 종로구,서울 마포구
3,2008,11590.0,서울,동작구,11590.0,서울,동작구,2,64,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구,서울 동작구
4,2008,11560.0,서울,영등포구,11560.0,서울,영등포구,1,72,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,36110.0,세종,세종시,1,80,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시
324127,2019,43112.0,충북,청주시 서원구,44230.0,충남,논산시,1,60,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구,충남 논산시
324128,2019,36110.0,세종,세종시,36110.0,세종,세종시,2,56,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시
324129,2019,36110.0,세종,세종시,36110.0,세종,세종시,2,35,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,세종 세종시


In [252]:
ohca_all['PRE_ER_ARREST_DT'].dtype

dtype('O')

In [245]:
# PRE_ER_ARREST_DT == 99999999 면 대신 nan 넣어주기

ohca_all.loc[ohca_all['PRE_ER_ARREST_DT'] == '99999999', 'PRE_ER_ARREST_DT'] = np.nan

In [246]:
# PRE_ER_ARREST_DT ends with 99 면 대신 nan 넣어주기

ohca_all.loc[ohca_all['PRE_ER_ARREST_DT'].str[6:8] == '99', 'PRE_ER_ARREST_DT'] = np.nan

In [247]:
ohca_all['PRE_ER_ARREST_DT'] = ohca_all['PRE_ER_ARREST_DT'].astype(object)

In [248]:
# 다시 nan check

ohca_all['PRE_ER_ARREST_DT'].isna().sum()

0

In [167]:
# PRE_ER_ARREST_DT 결측 (187,122) -> LAST_NORM_DT 로 채우기

ohca_all['PRE_ER_ARREST_DT'].fillna(ohca_all['LAST_NORM_DT'], inplace = True)

In [168]:
# 기존 18만개 결측 -> 47000개 결측으로 줄음

ohca_all['PRE_ER_ARREST_DT'].isna().sum()

47413

In [169]:
# 그래도 결측 있음 -> 47000개 결측 ER_ADM_DT 로 채우기

ohca_all['PRE_ER_ARREST_DT'].fillna(ohca_all['ER_ADM_DT'], inplace = True)

In [170]:
# 이제 결측 없음!

ohca_all['PRE_ER_ARREST_DT'].isna().sum()

0

In [172]:
# P_ADMINCODE NA row check 

ohca_all.loc[ohca_all['P_ADMINCODE'].isna() == True]

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,...,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST,P_ADD_CITY_DIST
42,2008,11680.0,서울,강남구,NaN,미상,미상,2,80,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구,미상 미상
46,2008,11680.0,서울,강남구,NaN,미상,미상,1,60,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구,미상 미상
53,2008,11710.0,서울,송파구,NaN,미상,미상,1,31,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 송파구,미상 미상
79,2008,11680.0,서울,강남구,NaN,미상,미상,2,83,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구,미상 미상
82,2008,11710.0,서울,송파구,NaN,미상,미상,1,18,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 송파구,미상 미상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323911,2019,50110.0,제주,제주시,NaN,미상,미상,2,92,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주 제주시,미상 미상
323912,2019,50110.0,제주,제주시,NaN,미상,미상,1,89,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주 제주시,미상 미상
323923,2019,50110.0,제주,제주시,NaN,미상,미상,2,79,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주 제주시,미상 미상
323926,2019,50110.0,제주,제주시,NaN,미상,미상,1,76,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제주 제주시,미상 미상


In [173]:
# H_는 있는데 P_는 없는 경우가 많음. ADMINCODE 및 일자의 NaN 처리를 위해 hospital, patient 테이블을 아예 분리해놓고 시작하자. 

##### ohca_hospital, ohca_patient 생성

In [174]:
ohca_hospital = ohca_all.drop(['P_ADMINCODE', 'P_ADD_CITY', 'P_ADD_DIST', 'P_ADD_CITY_DIST'], axis = 1)
ohca_patient = ohca_all.drop(['H_ADMINCODE', 'H_ADD_CITY', 'H_ADD_DIST', 'H_ADD_CITY_DIST'], axis = 1)

In [175]:
ohca_hospital

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST
0,2008,11410.0,서울,서대문구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서대문구
1,2008,11320.0,서울,도봉구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구
2,2008,11110.0,서울,종로구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 종로구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,43112.0,충북,청주시 서원구,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


In [176]:
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11680.0,서울,강남구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
1,2008,11305.0,서울,강북구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
2,2008,11440.0,서울,마포구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 마포구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,44230.0,충남,논산시,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


#### 결측 처리 및 전처리                 

0. 특이사항 수기 수정          
    - ohca_hospital['H_ADD_CITY_DIST'] == '부산 진구' 에 대한 H_ADMINCODE missing, 찾아서 넣어줌 (행정표준코드관리시스템)
    - '부산 진구' -> '부산 부산진구' 로 명칭 변경 (이게 정식 명칭이고, ohca_patient 에는 '부산 부산진구' 로 되어있음, 이래서 결측이 생겼던 것!)
    - ohca_patient['P_ADD_CITY_DIST'] == '충남 굼산군' --> '충남 금산군' 으로 수정     
    - '충남 금산군' 코드 찾아서 'P_ADMINCODE' 에 넣어줌 (행정표준코드관리시스템)               
                                                         
                                                   
                                                         
1. H_ADMINCODE, P_ADMINCODE 결측 drop    
                                                                 
                                                                                                                                       
2. PRE_ER_ARREST_DT 결측 drop                                                          

                                                                                                    
3. H_ADD_CITY_DIST, P_ADD_CITY_DIST endswith '미상' drop    


4. 'PRE_ER_ARREST_DT' 처리       
    - 99999999.0         
    - 20081299.0           

##### 0. 특이사항 수기 수정  

In [177]:
# ohca_hospital['H_ADD_CITY_DIST'] == '부산 진구' 에 대한 H_ADMINCODE missing, 찾아서 넣어줌 (행정표준코드관리시스템)
# '부산 진구' -> '부산 부산진구' 로 명칭 변경 (이게 정식 명칭이고, ohca_patient 에는 '부산 부산진구' 로 되어있음, 이래서 결측이 생겼던 것!) 

ohca_hospital.loc[ohca_hospital['H_ADD_CITY_DIST'] == '부산 진구', 'H_ADMINCODE'] = 26230.0
ohca_hospital.loc[ohca_hospital['H_ADD_CITY_DIST'] == '부산 진구', 'H_ADD_CITY_DIST'] = '부산 부산진구'
ohca_hospital

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST
0,2008,11410.0,서울,서대문구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서대문구
1,2008,11320.0,서울,도봉구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구
2,2008,11110.0,서울,종로구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 종로구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,43112.0,충북,청주시 서원구,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


In [178]:
# 혹시 모르니까 ohca_patient 에도 넣어줌

ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'] == '부산 진구', 'P_ADMINCODE'] = 26230.0
ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'] == '부산 진구', 'P_ADD_CITY_DIST'] = '부산 부산진구'
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11680.0,서울,강남구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
1,2008,11305.0,서울,강북구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
2,2008,11440.0,서울,마포구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 마포구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,44230.0,충남,논산시,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


In [179]:
# ohca_patient['P_ADD_CITY_DIST'] == '충남 굼산군' --> '충남 금산군' 으로 수정
# '충남 금산군' 코드 찾아서 'P_ADMINCODE' 에 넣어줌 (행정표준코드관리시스템)  

ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'] == '충남 굼산군']

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
285281,2018,NaN,충남,굼산군,1,66,1,2,20180707.0,1799.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 굼산군


In [180]:
# ohca_hospital에는 없음

ohca_hospital.loc[ohca_hospital['H_ADD_CITY_DIST'] == '충남 굼산군']

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST


In [181]:
ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'] == '충남 굼산군', 'P_ADD_CITY_DIST'] = '충남 금산군'
ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'] == '충남 금산군', 'P_ADMINCODE'] = 44710.0
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11680.0,서울,강남구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
1,2008,11305.0,서울,강북구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
2,2008,11440.0,서울,마포구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 마포구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,44230.0,충남,논산시,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


##### 1. H_ADMINCODE, P_ADMINCODE 결측 drop

In [182]:
ohca_hospital.loc[ohca_hospital['PRE_ER_ARREST_DT'].isna() == True]

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST


In [183]:
ohca_hospital.dropna(subset=['H_ADMINCODE'], inplace = True)
ohca_patient.dropna(subset=['P_ADMINCODE'], inplace = True)

##### 2. PRE_ER_ARREST_DT 결측 drop

In [184]:
ohca_hospital.dropna(subset=['PRE_ER_ARREST_DT'], inplace = True)
ohca_patient.dropna(subset=['PRE_ER_ARREST_DT'], inplace = True)

##### 3. H_ADD_CITY_DIST, P_ADD_CITY_DIST endswith '미상' drop

In [185]:
# ohca_hospital에는 없음

ohca_hospital.loc[ohca_hospital['H_ADD_CITY_DIST'].str.endswith('미상') == True]

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST


In [186]:
# ohca_patient 확인 -> 있음

ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'].str.endswith('미상') == True]

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
1523,2008,41000.0,경기,미상,1,69,1,2,20080711.0,2135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,경기 미상
2046,2008,11000.0,서울,미상,1,26,2,2,20080915.0,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 미상
14194,2008,41000.0,경기,미상,1,45,7,1,20080624,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,경기 미상
14347,2008,44000.0,충남,미상,2,78,1,1,20081116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 미상
24386,2009,48000.0,경남,미상,1,19,1,2,20090710.0,1609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,경남 미상
29731,2009,41000.0,경기,미상,1,86,9,2,20090307.0,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,경기 미상
34318,2009,27000.0,대구,미상,1,41,1,2,20090112.0,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대구 미상
35997,2009,44000.0,충남,미상,1,53,1,2,20090513.0,1644.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 미상
47408,2010,26000.0,부산,미상,2,51,1,1,20100623,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,부산 미상
54537,2010,11000.0,서울,미상,1,53,1,2,20100628.0,1046.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 미상


In [187]:
# ohca_patient endswith '미상' drop

ohca_patient = ohca_patient.loc[ohca_patient['P_ADD_CITY_DIST'].str.endswith('미상') == False].copy()
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11680.0,서울,강남구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
1,2008,11305.0,서울,강북구,2,47,1,2,20080101.0,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
2,2008,11440.0,서울,마포구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 마포구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101.0,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,44230.0,충남,논산시,1,60,3,2,20191004.0,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


##### 'PRE_ER_ARREST_DT' 처리

In [191]:
# 'PRE_ER_ARREST_DT' 20080101.0 형식에서 .0 자르기

ohca_patient['PRE_ER_ARREST_DT']= ohca_patient['PRE_ER_ARREST_DT'].str[0:8]
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11680.0,서울,강남구,1,58,1,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
1,2008,11305.0,서울,강북구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
2,2008,11440.0,서울,마포구,2,81,5,9,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 마포구
3,2008,11590.0,서울,동작구,2,64,9,1,20080101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 동작구
4,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324126,2019,36110.0,세종,세종시,1,80,1,1,20191127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324127,2019,44230.0,충남,논산시,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시
324128,2019,36110.0,세종,세종시,2,56,9,1,20191116,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
324129,2019,36110.0,세종,세종시,2,35,9,1,20191024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시


In [90]:
# 99999999 제거 (40개)

ohca_hospital = ohca_hospital.loc[ohca_hospital['PRE_ER_ARREST_DT'] != '99999999'].reset_index(drop = True)
ohca_hospital

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST
0,2008,11320.0,서울,도봉구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
2,2008,11680.0,서울,강남구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136784,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
136785,2019,36110.0,세종,세종시,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
136786,2019,30170.0,대전,서구,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대전 서구
136787,2019,43112.0,충북,청주시 서원구,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구


In [91]:
# 99999999 제거 (56개)

ohca_patient = ohca_patient.loc[ohca_patient['PRE_ER_ARREST_DT'] != '99999999'].reset_index(drop = True)
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11305.0,서울,강북구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
2,2008,11650.0,서울,서초구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133463,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
133464,2019,11350.0,서울,노원구,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 노원구
133465,2019,36110.0,세종,세종시,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
133466,2019,44230.0,충남,논산시,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시


In [92]:
# 99일 처리된 일자 (139개) -> 제거

ohca_hospital = ohca_hospital.loc[ohca_hospital['PRE_ER_ARREST_DT'].str[6:8] != '99'].reset_index(drop = True)
ohca_hospital

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST
0,2008,11320.0,서울,도봉구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
2,2008,11680.0,서울,강남구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136629,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
136630,2019,36110.0,세종,세종시,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
136631,2019,30170.0,대전,서구,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대전 서구
136632,2019,43112.0,충북,청주시 서원구,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구


In [93]:
# 99일 처리된 일자 (138개) -> 제거

ohca_patient = ohca_patient.loc[ohca_patient['PRE_ER_ARREST_DT'].str[6:8] != '99'].reset_index(drop = True)
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_RR,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST
0,2008,11305.0,서울,강북구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구
2,2008,11650.0,서울,서초구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133317,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
133318,2019,11350.0,서울,노원구,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 노원구
133319,2019,36110.0,세종,세종시,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시
133320,2019,44230.0,충남,논산시,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시


In [94]:
ohca_patient.reset_index(drop = True, inplace = True)
ohca_hospital.reset_index(drop = True, inplace = True)

In [95]:
# 결측 다시 확인

ohca_hospital.isna().sum()

YEAR                    0
H_ADMINCODE             0
H_ADD_CITY              0
H_ADD_DIST              0
H_SEX                   0
                    ...  
OP8_ROSC           136252
OP8_ROSC_DT        136302
OP8_ROSC_TM        136302
BYSTANDER_AED      136571
H_ADD_CITY_DIST         0
Length: 229, dtype: int64

In [96]:
# 결측 다시 확인

ohca_patient.isna().sum()

YEAR                    0
P_ADMINCODE             0
P_ADD_CITY              0
P_ADD_DIST              0
H_SEX                   0
                    ...  
OP8_ROSC           132946
OP8_ROSC_DT        132993
OP8_ROSC_TM        132993
BYSTANDER_AED      133264
P_ADD_CITY_DIST         0
Length: 229, dtype: int64

##### 결측 처리 및 전처리 완.

---------------------------------------------------

#### 일자별 x 시군구 cross join index 생성

##### 임의의 환자 고유 아이디 생성 및 연간 발생 case count

In [97]:
# 임의의 환자 고유 아이디 생성 (NUM)

ohca_hospital['NUM'] = np.arange(len(ohca_hospital))
ohca_hospital

,YEAR,H_ADMINCODE,H_ADD_CITY,H_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,H_ADD_CITY_DIST,NUM
0,2008,11320.0,서울,도봉구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 도봉구,0
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구,1
2,2008,11680.0,서울,강남구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강남구,2
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구,3
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136629,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,136629
136630,2019,36110.0,세종,세종시,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,136630
136631,2019,30170.0,대전,서구,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대전 서구,136631
136632,2019,43112.0,충북,청주시 서원구,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충북 청주시 서원구,136632


In [98]:
# 병원 소재지 기준 연간 30000명 가량 (원시자료 설명 기준) -> 11386명으로 줄음

136634/12

11386.166666666666

In [99]:
# 임의의 환자 고유 아이디 생성 (NUM)

ohca_patient['NUM'] = np.arange(len(ohca_patient))
ohca_patient

,YEAR,P_ADMINCODE,P_ADD_CITY,P_ADD_DIST,H_SEX,AGE,INS,WITNESS,PRE_ER_ARREST_DT,PRE_ER_ARREST_TM,...,OP8_PRE_VITAL_BT,OP8_PRE_VITAL_DT,OP8_PRE_VITAL_TM,OP8_SUCCESS,OP8_ROSC,OP8_ROSC_DT,OP8_ROSC_TM,BYSTANDER_AED,P_ADD_CITY_DIST,NUM
0,2008,11305.0,서울,강북구,2,47,1,2,20080101,1000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구,0
1,2008,11560.0,서울,영등포구,1,72,1,2,20080101,1405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 영등포구,1
2,2008,11650.0,서울,서초구,1,54,1,2,20080101,1745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구,2
3,2008,11650.0,서울,서초구,1,48,9,2,20080101,1930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 서초구,3
4,2008,11305.0,서울,강북구,1,70,9,2,20080102,1114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 강북구,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133317,2019,36110.0,세종,세종시,1,77,1,2,20191202,930.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,133317
133318,2019,11350.0,서울,노원구,1,52,1,2,20191203,1047.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울 노원구,133318
133319,2019,36110.0,세종,세종시,1,0,1,2,20191219,9999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,세종 세종시,133319
133320,2019,44230.0,충남,논산시,1,60,3,2,20191004,1337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,충남 논산시,133320


In [100]:
# 환자 소재지 기준 연간 30000명 가량 (원시자료 설명 기준) -> 11110명으로 줄음 

133322 / 12

11110.166666666666

##### 2008 ~ 2020-01-01까지의 모든 일자 dataframe으로 만들기

In [101]:
# 2008 ~ 2020-01-01까지의 모든 일자 dataframe으로 만들기

rng = pd.date_range(start = '2008-01-01', end = '2020-01-01', freq='D')
days = pd.DataFrame({'PRE_ER_ARREST_DT': rng}) 
days

,PRE_ER_ARREST_DT
0,2008-01-01
1,2008-01-02
2,2008-01-03
3,2008-01-04
4,2008-01-05
...,...
4379,2019-12-28
4380,2019-12-29
4381,2019-12-30
4382,2019-12-31


##### hospital_cityname_index, patient_cityname_index 만들기

In [102]:
# 전처리 완료된 ohca_hospital에서 admincode, H_ADD_CITY_DIST 만 뽑아와서 unique, drop dup
# 총 253개 index

hospital_cityname_index = ohca_hospital[['H_ADMINCODE', 'H_ADD_CITY_DIST']].copy()
hospital_cityname_index.drop_duplicates(subset=['H_ADMINCODE', 'H_ADD_CITY_DIST'], inplace = True)
hospital_cityname_index.reset_index(drop = True, inplace = True)
hospital_cityname_index

,H_ADMINCODE,H_ADD_CITY_DIST
0,11320.0,서울 도봉구
1,11560.0,서울 영등포구
2,11680.0,서울 강남구
3,11650.0,서울 서초구
4,11305.0,서울 강북구
...,...,...
248,41190.0,경기 부천시
249,28177.0,인천 미추홀구
250,41670.0,경기 여주시
251,43112.0,충북 청주시 서원구


In [104]:
print(len(hospital_cityname_index['H_ADD_CITY_DIST'].unique()))
print(len(hospital_cityname_index['H_ADMINCODE'].unique()))   

# 갯수 안 맞음. 
# (코드, 시군구 이름) 기준으로 drop dup 했는데 안 맞는다는 건 코드는 같은데 이름이 다른 경우가 있다는 뜻, 내보내서 확인 한번 하자
# 부산 부산진구 != 부산 진구 이거였음

# ---> 맞췄음!

253
253


In [105]:
# hospital_cityname_index.to_csv('data\\dump\\hospital_cityname_index.csv', encoding = 'euc-kr', index = False)

In [106]:
# 전처리 완료된 ohca_patient에서 admincode, P_ADD_CITY_DIST 만 뽑아와서 unique, drop dup
# 총 263개 index

patient_cityname_index = ohca_patient[['P_ADMINCODE', 'P_ADD_CITY_DIST']].copy()
patient_cityname_index.drop_duplicates(subset=['P_ADMINCODE', 'P_ADD_CITY_DIST'], inplace = True)
patient_cityname_index.reset_index(drop = True, inplace = True)
patient_cityname_index

,P_ADMINCODE,P_ADD_CITY_DIST
0,11305.0,서울 강북구
1,11560.0,서울 영등포구
2,11650.0,서울 서초구
3,11530.0,서울 구로구
4,11230.0,서울 동대문구
...,...,...
258,43112.0,충북 청주시 서원구
259,41190.0,경기 부천시
260,28177.0,인천 미추홀구
261,43110.0,충북 청주시


In [107]:
patient_cityname_index.loc[patient_cityname_index['P_ADMINCODE'] == 26230]

,P_ADMINCODE,P_ADD_CITY_DIST
49,26230.0,부산 부산진구


In [108]:
# 갯수 일치 확인

print(len(patient_cityname_index['P_ADD_CITY_DIST'].unique()))
print(len(patient_cityname_index['P_ADMINCODE'].unique()))   

263
263


In [109]:
# patient_cityname_index P_ADMINCODE 결측 (당연히) 없음

patient_cityname_index.isna().sum()

P_ADMINCODE        0
P_ADD_CITY_DIST    0
dtype: int64

In [110]:
# hospital_cityname_index H_ADMINCODE 결측 (당연히) 없음

hospital_cityname_index.isna().sum()

H_ADMINCODE        0
H_ADD_CITY_DIST    0
dtype: int64

In [111]:
hospital_cityname_index.dtypes

H_ADMINCODE        float64
H_ADD_CITY_DIST     object
dtype: object

In [112]:
# # 내보내서 뭐 있는지 한번 확인

# hospital_cityname_index.to_csv('data\\급성심장정지조사(08-19)SAS\\hospital_cityname_index.csv', encoding = 'euc-kr', index = False)
# patient_cityname_index.to_csv('data\\급성심장정지조사(08-19)SAS\\patient_cityname_index.csv', encoding = 'euc-kr', index = False)

##### 2008-2020.01.01 days x cityname_index cross join 해서 일자별, 시군구 index 생성하기

In [113]:
# cross join days x code

hospital_index = days.merge(hospital_cityname_index, how='cross')
hospital_index

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST
0,2008-01-01,11320.0,서울 도봉구
1,2008-01-01,11560.0,서울 영등포구
2,2008-01-01,11680.0,서울 강남구
3,2008-01-01,11650.0,서울 서초구
4,2008-01-01,11305.0,서울 강북구
...,...,...,...
1109147,2020-01-01,41190.0,경기 부천시
1109148,2020-01-01,28177.0,인천 미추홀구
1109149,2020-01-01,41670.0,경기 여주시
1109150,2020-01-01,43112.0,충북 청주시 서원구


In [119]:
# hospital_index 일수, admincode 수 확인 - 맞음

print(len(hospital_index['PRE_ER_ARREST_DT'].unique()))
print(len(hospital_index['H_ADMINCODE'].unique()))
print(len(hospital_index['H_ADD_CITY_DIST'].unique()))

4384
253
253


In [121]:
# 개수 맞음!

4384 * 253

1109152

In [118]:
# cross join days x code

patient_index = days.merge(patient_cityname_index, how='cross')
patient_index

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST
0,2008-01-01,11305.0,서울 강북구
1,2008-01-01,11560.0,서울 영등포구
2,2008-01-01,11650.0,서울 서초구
3,2008-01-01,11530.0,서울 구로구
4,2008-01-01,11230.0,서울 동대문구
...,...,...,...
1152987,2020-01-01,43112.0,충북 청주시 서원구
1152988,2020-01-01,41190.0,경기 부천시
1152989,2020-01-01,28177.0,인천 미추홀구
1152990,2020-01-01,43110.0,충북 청주시


In [120]:
# patient_index 일수, admincode 수 확인 - 맞음

print(len(patient_index['PRE_ER_ARREST_DT'].unique()))
print(len(patient_index['P_ADMINCODE'].unique()))
print(len(patient_index['P_ADD_CITY_DIST'].unique()))

4384
263
263


In [123]:
# 개수 맞음 ! 

4384 * 263

1152992

##### 병원 소재지별, 급성심장정지 목격일자별 aggregate

In [124]:
# 병원 소재지별, 급성심장정지 목격일자별 aggregate

group_hospital = ohca_hospital.groupby(['PRE_ER_ARREST_DT', 'H_ADMINCODE', 'H_ADD_CITY_DIST'])
group_hospital = pd.DataFrame(group_hospital.size().reset_index(name='count'))

In [125]:
# check 

group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,20080101,11320.0,서울 도봉구,1
1,20080101,11560.0,서울 영등포구,1
2,20080101,11650.0,서울 서초구,1
3,20080101,11680.0,서울 강남구,1
4,20080101,26260.0,부산 동래구,1
...,...,...,...,...
119978,20191231,48123.0,경남 창원시 성산구,1
119979,20191231,48170.0,경남 진주시,1
119980,20191231,48330.0,경남 양산시,2
119981,20191231,50110.0,제주 제주시,1


In [126]:
# 환자 거주지별, 급성심장정지 목격일자별 aggregate

group_patient = ohca_patient.groupby(['PRE_ER_ARREST_DT', 'P_ADMINCODE', 'P_ADD_CITY_DIST'])
group_patient = pd.DataFrame(group_patient.size().reset_index(name='count'))

In [127]:
# check 

group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,20080101,11305.0,서울 강북구,1
1,20080101,11500.0,서울 강서구,1
2,20080101,11560.0,서울 영등포구,1
3,20080101,11650.0,서울 서초구,2
4,20080101,26260.0,부산 동래구,1
...,...,...,...,...
121681,20191231,48123.0,경남 창원시 성산구,1
121682,20191231,48240.0,경남 사천시,1
121683,20191231,48330.0,경남 양산시,1
121684,20191231,50130.0,제주 서귀포시,1


##### PRE_ER_ARREST_DT 처리 및 정렬

In [128]:
# group_hospital

group_hospital['PRE_ER_ARREST_DT'] = pd.to_datetime(group_hospital['PRE_ER_ARREST_DT'])
group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,2008-01-01,11320.0,서울 도봉구,1
1,2008-01-01,11560.0,서울 영등포구,1
2,2008-01-01,11650.0,서울 서초구,1
3,2008-01-01,11680.0,서울 강남구,1
4,2008-01-01,26260.0,부산 동래구,1
...,...,...,...,...
119978,2019-12-31,48123.0,경남 창원시 성산구,1
119979,2019-12-31,48170.0,경남 진주시,1
119980,2019-12-31,48330.0,경남 양산시,2
119981,2019-12-31,50110.0,제주 제주시,1


In [129]:
# group_patient

group_patient['PRE_ER_ARREST_DT'] = pd.to_datetime(group_patient['PRE_ER_ARREST_DT'])
group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,2008-01-01,11305.0,서울 강북구,1
1,2008-01-01,11500.0,서울 강서구,1
2,2008-01-01,11560.0,서울 영등포구,1
3,2008-01-01,11650.0,서울 서초구,2
4,2008-01-01,26260.0,부산 동래구,1
...,...,...,...,...
121681,2019-12-31,48123.0,경남 창원시 성산구,1
121682,2019-12-31,48240.0,경남 사천시,1
121683,2019-12-31,48330.0,경남 양산시,1
121684,2019-12-31,50130.0,제주 서귀포시,1


In [131]:
# date로 sort

group_hospital.sort_values('PRE_ER_ARREST_DT', inplace = True)
group_hospital.reset_index(drop = True, inplace = True)

In [132]:
# check

group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST,count
0,2008-01-01,11320.0,서울 도봉구,1
1,2008-01-01,50110.0,제주 제주시,1
2,2008-01-01,48870.0,경남 함양군,1
3,2008-01-01,48250.0,경남 김해시,1
4,2008-01-01,48125.0,경남 창원시 마산합포구,1
...,...,...,...,...
119978,2019-12-31,31110.0,울산 중구,1
119979,2019-12-31,41135.0,경기 성남시 분당구,1
119980,2019-12-31,41273.0,경기 안산시 단원구,1
119981,2019-12-31,26350.0,부산 해운대구,1


In [133]:
# date로 sort

group_patient.sort_values('PRE_ER_ARREST_DT', inplace = True)
group_patient.reset_index(drop = True, inplace = True)

In [134]:
# check

group_patient

,PRE_ER_ARREST_DT,P_ADMINCODE,P_ADD_CITY_DIST,count
0,2008-01-01,11305.0,서울 강북구,1
1,2008-01-01,50110.0,제주 제주시,1
2,2008-01-01,48870.0,경남 함양군,1
3,2008-01-01,48250.0,경남 김해시,1
4,2008-01-01,48160.0,경남 마산시,1
...,...,...,...,...
121681,2019-12-31,30200.0,대전 유성구,1
121682,2019-12-31,31110.0,울산 중구,1
121683,2019-12-31,41131.0,경기 성남시 수정구,1
121684,2019-12-31,26260.0,부산 동래구,2


##### hospital_index, patient_index LEFT JOIN aggregated data

In [136]:
hospital_index

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST
0,2008-01-01,11320.0,서울 도봉구
1,2008-01-01,11560.0,서울 영등포구
2,2008-01-01,11680.0,서울 강남구
3,2008-01-01,11650.0,서울 서초구
4,2008-01-01,11305.0,서울 강북구
...,...,...,...
1109147,2020-01-01,41190.0,경기 부천시
1109148,2020-01-01,28177.0,인천 미추홀구
1109149,2020-01-01,41670.0,경기 여주시
1109150,2020-01-01,43112.0,충북 청주시 서원구


In [137]:
# hospital_index랑 left join 하기

group_hospital = pd.merge(hospital_index, group_hospital, on=['PRE_ER_ARREST_DT', 'H_ADMINCODE'], how = 'left')
group_hospital

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST_x,H_ADD_CITY_DIST_y,count
0,2008-01-01,11320.0,서울 도봉구,서울 도봉구,1.0
1,2008-01-01,11560.0,서울 영등포구,서울 영등포구,1.0
2,2008-01-01,11680.0,서울 강남구,서울 강남구,1.0
3,2008-01-01,11650.0,서울 서초구,서울 서초구,1.0
4,2008-01-01,11305.0,서울 강북구,NaN,NaN
...,...,...,...,...,...
1109147,2020-01-01,41190.0,경기 부천시,NaN,NaN
1109148,2020-01-01,28177.0,인천 미추홀구,NaN,NaN
1109149,2020-01-01,41670.0,경기 여주시,NaN,NaN
1109150,2020-01-01,43112.0,충북 청주시 서원구,NaN,NaN


In [138]:
group_hospital.head(50)

,PRE_ER_ARREST_DT,H_ADMINCODE,H_ADD_CITY_DIST_x,H_ADD_CITY_DIST_y,count
0,2008-01-01,11320.0,서울 도봉구,서울 도봉구,1.0
1,2008-01-01,11560.0,서울 영등포구,서울 영등포구,1.0
2,2008-01-01,11680.0,서울 강남구,서울 강남구,1.0
3,2008-01-01,11650.0,서울 서초구,서울 서초구,1.0
4,2008-01-01,11305.0,서울 강북구,NaN,NaN
5,2008-01-01,11530.0,서울 구로구,NaN,NaN
6,2008-01-01,11290.0,서울 성북구,NaN,NaN
7,2008-01-01,11470.0,서울 양천구,NaN,NaN
8,2008-01-01,11260.0,서울 중랑구,NaN,NaN
9,2008-01-01,11110.0,서울 종로구,NaN,NaN


In [ ]:
# hospital_index랑 left join 하기

group_hospital = pd.merge(hospital_index, group_hospital, on=['PRE_ER_ARREST_DT', 'H_ADMINCODE'], how = 'left')
group_hospital

In [ ]:
# 지역코드별 count 몇명인지 한번 check
# group_hospital

hospital_tmp = group_hospital.groupby(['H_ADMINCODE', 'H_ADD_CITY_DIST']).count()
hospital_tmp.sort_values(by='count', ascending = False, inplace = True)
hospital_tmp

In [ ]:
# 지역코드별 count 몇명인지 한번 check
# group_patient

patient_tmp = group_patient.groupby(['P_ADMINCODE', 'P_ADD_CITY_DIST']).count()
patient_tmp.sort_values(by='count', ascending = False, inplace = True)
patient_tmp

In [ ]:
# 두 테이블 다 ADMINCODE 숫자로 바꾸기

group_hospital['H_ADMINCODE'] = group_hospital['H_ADMINCODE'].astype(int)
group_patient['P_ADMINCODE'] = group_patient['P_ADMINCODE'].astype(int)

In [ ]:
group_hospital

In [ ]:
group_patient

##### qgis 지도 idx       
- SIGUNGU_CD : 원래 유진선생님이 주신 raw data의 시군구 코드
- 법정동코드 : 행정표준코드관리시스템 기준 법정동코드, 급성심정지 데이터는 이 기준으로 되어 있음

In [ ]:
# qgis 지도 idx 불러오기

idx = pd.read_csv('data\\out of hospital sudden cardiac arrest\\qgis_지도_idx.csv', encoding = 'euc-kr')
idx

In [ ]:
# 지도 idx table이랑 시군구 코드 column 명 맞추기

group_hospital.rename(columns={'H_ADMINCODE' : '법정동코드'}, inplace = True)
group_patient.rename(columns={'P_ADMINCODE' : '법정동코드'}, inplace = True)

In [ ]:
group_hospital

In [ ]:
# Merge group_hospital with idx with 법정동코드

group_hospital_idx = pd.merge(group_hospital, idx, on='법정동코드', how='inner')
group_hospital_idx.sort_values(by='PRE_ER_ARREST_DT', inplace = True)
group_hospital_idx.reset_index(drop = True, inplace = True)
group_hospital_idx

In [ ]:
# Merge group_patient with idx with 법정동코드

group_patient_idx = pd.merge(group_patient, idx, on='법정동코드', how='inner')
group_patient_idx.sort_values(by='PRE_ER_ARREST_DT', inplace = True)
group_patient_idx.reset_index(drop = True, inplace = True)
group_patient_idx

In [ ]:
group_hospital_idx.isna().sum()

In [ ]:
group_patient_idx.isna().sum()

In [ ]:
# aggregate by hospital location

#pd.set_option("display.max_rows", 253)
pd.reset_option('display.max_rows')
group_hospital_table_location = group_hospital_idx.pivot_table(values='count', index=['법정동코드', 'SIGUNGU_NM', 'H_ADD_CITY_DIST'], aggfunc=np.sum)
group_hospital_table_location

In [ ]:
# aggregate by patient location

group_patient_table_location = group_patient_idx.pivot_table(values='count', index=['법정동코드', 'SIGUNGU_NM', 'P_ADD_CITY_DIST'], aggfunc=np.sum)
group_patient_table_location

In [ ]:
# aggregate by dates

group_hospital_table_date = group_hospital_idx.pivot_table(values='count', index=['PRE_ER_ARREST_DT'], aggfunc=np.sum)
group_hospital_table_date

In [ ]:
# aggregate by dates

group_patient_table_date = group_patient_idx.pivot_table(values='count', index=['PRE_ER_ARREST_DT'], aggfunc=np.sum)
group_patient_table_date

In [ ]:
group_patient_idx.to_csv('data\급성심장정지조사(08-19)SAS\group_patient_idx.csv', encoding = 'euc-kr', index = False)
group_hospital_idx.to_csv('data\급성심장정지조사(08-19)SAS\group_hospital_idx.csv', encoding = 'euc-kr', index = False)

In [ ]:
# 지역별 aggregate 

group_patient_table_location.to_csv('data\급성심장정지조사(08-19)SAS\group_patient_table_location.csv', encoding = 'euc-kr')
group_hospital_table_location.to_csv('data\급성심장정지조사(08-19)SAS\group_hospital_table_location.csv', encoding = 'euc-kr')

In [ ]:
# 일자별 aggregate

group_patient_table_date.to_csv('data\급성심장정지조사(08-19)SAS\group_patient_table_date.csv', encoding = 'euc-kr')
group_hospital_table_date.to_csv('data\급성심장정지조사(08-19)SAS\group_hospital_table_date.csv', encoding = 'euc-kr')

#### Air data

##### index 생성

In [ ]:
# r 코드 돌려서 만든 air_2020_idx 가져오기

air_2020_idx_zio = pd.read_csv('data\\out of hospital sudden cardiac arrest\\air_2020_idx_zio.csv', encoding = 'euc-kr')
air_2020_idx_zio

In [ ]:
# merge with qgis_지도_idx

map_air_idx = pd.merge(idx, air_2020_idx_zio, left_on='OBJECTID(air_out_idx)', right_on='air_out_idx')
map_air_idx

In [ ]:
map_air_idx.drop_duplicates(subset=['air_out_idx'], inplace = True)
map_air_idx.reset_index(drop = True, inplace = True)
map_air_idx

In [ ]:
# 겹치는 column drop

map_air_idx.drop(columns = {'SIGUNGU_CD', 'SIGUNGU_NM', 'OBJECTID(air_out_idx)', 'check(수도권=1)', 'location', 'SOCIAL'}, inplace = True)
map_air_idx

In [ ]:
group_hospital_idx

In [ ]:
# merge group_hospital_idx with air, map idx

group_hospital_idx_air = pd.merge(group_hospital_idx, map_air_idx, on='법정동코드', how='left')
group_hospital_idx_air

In [ ]:
# merge group_patient_idx with air, map idx

group_patient_idx_air = pd.merge(group_patient_idx, map_air_idx, on='법정동코드', how='left')
group_patient_idx_air

In [ ]:
# 필요 없는 칼럼 한번 다시 정리

group_hospital_idx_air.drop(columns = {'SIGUNGU_NM', 'OBJECTID(air_out_idx)', 'location', 'SOCIAL'}, axis = 1, inplace = True)
group_patient_idx_air.drop(columns = {'SIGUNGU_NM', 'OBJECTID(air_out_idx)', 'location', 'SOCIAL'}, axis = 1, inplace = True)

In [ ]:
group_patient_idx_air

In [ ]:
len(group_patient_idx_air['측정소코드'].unique())

In [ ]:
len(group_patient_idx_air['air_out_idx'].unique())

In [ ]:
group_patient_idx_air['count'].isna().sum()

In [ ]:
len(group_hospital_idx_air['측정소코드'].unique())

In [ ]:
len(group_hospital_idx_air['air_out_idx'].unique())

In [ ]:
group_hospital_idx_air['count'].isna().sum()

##### air data 불러오기

In [ ]:
# air data 불러와서 합치기 - 11분 소요

year_list = np.arange(2008, 2020, 1).tolist()
year_list = list(map(str, year_list))
year_list

air_data = pd.DataFrame()
for year in year_list: 
    air_tmp = pd.read_excel('data\\out of hospital sudden cardiac arrest\\air_year_{}.xlsx'.format(year))
    air_data = pd.concat([air_data, air_tmp], axis = 0)

In [ ]:
# air_data.to_csv('data\\out of hospital sudden cardiac arrest\\air_0819_zio.csv', encoding = 'euc-kr', index = False)

In [ ]:
# air_data 다시 불러오기 

air_data = pd.read_csv('data\\out of hospital sudden cardiac arrest\\air_0819_zio.csv', encoding = 'euc-kr')

In [ ]:
air_data

In [ ]:
# '주소' 겹치니까 지우기

air_data.drop(columns = ['주소'], axis = 1, inplace = True)
air_data

In [ ]:
air_data.isna().sum()

In [ ]:
len(air_data['측정소코드'].unique())

In [ ]:
air_data.dtypes

In [ ]:
# date column 이름 맞춰주기

air_data.rename(columns = {'dt' : 'PRE_ER_ARREST_DT'}, inplace = True)

In [ ]:
air_data['PRE_ER_ARREST_DT'].isna().sum()

In [ ]:
air_data['PRE_ER_ARREST_DT'] = pd.to_datetime(air_data['PRE_ER_ARREST_DT'])

In [ ]:
air_hospital_final = pd.merge(group_hospital_idx_air, air_data, on=['측정소코드', 'PRE_ER_ARREST_DT'], how = 'left')
air_hospital_final

In [ ]:
air_patient_final = pd.merge(group_patient_idx_air, air_data, on=['측정소코드', 'PRE_ER_ARREST_DT'], how = 'left')
air_patient_final

In [ ]:
len(air_hospital_final['air_out_idx'].unique())

In [ ]:
len(air_hospital_final['측정소코드'].unique())

In [ ]:
len(air_patient_final['air_out_idx'].unique())

In [ ]:
len(air_patient_final['측정소코드'].unique())

#### Weather data

##### weather index 불러오기
- 수도권 71개만 있음 (air_out_idx)

In [ ]:
weather_index = pd.read_csv('data\\out of hospital sudden cardiac arrest\\weather_index.csv', encoding = 'euc-kr')
weather_index

In [ ]:
weather_index.drop_duplicates(subset=['지점'], inplace = True)
weather_index.reset_index(drop = True, inplace = True)
weather_index

In [ ]:
weather_index.drop(['지점명'], axis = 1, inplace = True)

In [ ]:
weather_index

In [ ]:
len(weather_index['air_out_idx'].unique())

In [ ]:
# weather data 불러오기 - 11분 소요

year_list = np.arange(2008, 2020, 1).tolist()
year_list = list(map(str, year_list))
year_list

weather_data = pd.DataFrame()
for year in year_list: 
    weather_tmp = pd.read_excel('data\\out of hospital sudden cardiac arrest\\weather_day_year{}.xlsx'.format(year))
    weather_data = pd.concat([weather_data, weather_tmp], axis = 0)

In [ ]:
# weather_data.to_csv('data\\out of hospital sudden cardiac arrest\\weather_0819_zio.csv', encoding = 'euc-kr', index = False)

In [ ]:
weather_data = pd.read_csv('data\\out of hospital sudden cardiac arrest\\weather_0819_zio.csv', encoding = 'euc-kr')

In [ ]:
weather_data

In [ ]:
len(weather_data['지점'].unique())

In [ ]:
# weather_data.loc[weather_data['지점'] == 418, '지점명'].unique()

In [ ]:
# len(weather_data['지점'].unique())
# len(weather_index['지점'].unique())
# len(weather_index['air_out_idx'].unique())
# a = set(weather_data['지점'].unique())
# b = set(weather_index['지점'].unique())

# len(a.intersection(b))
# a2 = set(weather_data['지점명'].unique())
# b2 = set(weather_index['지점명'].unique())

# a2.intersection(b2)
# unique_place = weather_data['지점명'].unique().tolist()
# unique_place

In [ ]:
# pd.set_option("display.max_rows", 253)
weather_data_idx = pd.merge(weather_index, weather_data, on='지점', how = 'left')
weather_data_idx

In [ ]:
weather_data_idx['지점'].isna().sum()

In [ ]:
weather_data_idx['air_out_idx'].isna().sum()

In [ ]:
len(weather_data_idx['air_out_idx'].unique())

In [ ]:
len(weather_data_idx['지점'].unique())

In [ ]:
# 열 이름 통일

weather_data_idx.rename(columns = {'dt': 'PRE_ER_ARREST_DT'}, inplace = True)
weather_data_idx

In [ ]:
# 날짜 기준으로 sort

weather_data_idx = weather_data_idx.sort_values(by=['PRE_ER_ARREST_DT'])
weather_data_idx.reset_index(drop = True, inplace = True)
weather_data_idx

In [ ]:
# pd.set_option('display.max_rows', 100)
# tmp = weather_data_idx.copy()
# tmp['PRE_ER_ARREST_DT'] = tmp['PRE_ER_ARREST_DT'].astype(str)
# tmp.groupby(tmp['PRE_ER_ARREST_DT'].str[0:4]).count()

In [ ]:
len(weather_data_idx['air_out_idx'].unique())

In [ ]:
# datetime 형식으로 바꾸기

weather_data_idx['PRE_ER_ARREST_DT'] = pd.to_datetime(weather_data_idx['PRE_ER_ARREST_DT'])
weather_data_idx

In [ ]:
weather_data_idx.isna().sum()

#### 최종 ohca+air+weather 데이터 생성

##### 일단 수도권만 확인

In [ ]:
a = set(air_hospital_final['air_out_idx'].unique())
b = set(weather_data_idx['air_out_idx'].unique())

len(a.intersection(b))

In [ ]:
a = set(air_patient_final['air_out_idx'].unique())
b = set(weather_data_idx['air_out_idx'].unique())

len(a.intersection(b))

In [ ]:
len(air_hospital_final['air_out_idx'].unique())

In [ ]:
air_hospital_final_sudogwon = air_hospital_final.loc[air_hospital_final['check(수도권=1)'] == 1].reset_index(drop = True)
air_hospital_final_sudogwon

In [ ]:
# 그냥 inner join 하는 게 맞겠다

hospital = pd.merge(air_hospital_final_sudogwon, weather_data_idx, on=['air_out_idx', 'PRE_ER_ARREST_DT'], how='inner')
hospital

In [ ]:
from datetime import timedelta

datetime.datetime(2019,12,31) - datetime.datetime(2008,1,1)

In [ ]:
len(hospital['air_out_idx'].unique())

In [ ]:
4382*57

In [ ]:
patient = pd.merge(air_patient_final, weather_data_idx, on=['air_out_idx', 'PRE_ER_ARREST_DT'], how='inner')
patient

In [ ]:
len(patient['air_out_idx'].unique())

In [ ]:
hospital['H_ADD_CITY_DIST'].unique()

In [ ]:
patient['P_ADD_CITY_DIST'].unique()

In [ ]:
hospital.columns

In [ ]:
# 필요 없는 칼럼 drop

hospital.drop(['지역', '측정소명', '지점', '지점명'], axis = 1, inplace = True)
patient.drop(['지역', '측정소명', '지점', '지점명'], axis = 1, inplace = True)

In [ ]:
len(hospital['air_out_idx'].unique())

In [ ]:
len(patient['air_out_idx'].unique())

In [ ]:
hospital.columns

In [ ]:
# hospital, patient air_out_idx 개수 71개 안됨 (inner join 할 시)
# --> 그냥 NaN 신경 안쓰고 air_out_idx 기준으로 right join 했음 (71개 다 살려보자)
# --> 다시 inner join 으로 돌아감

In [ ]:
hospital.drop_duplicates(subset=['air_out_idx', 'PRE_ER_ARREST_DT'], inplace = True)
hospital.reset_index(drop = True, inplace = True)
hospital

In [ ]:
patient.drop_duplicates(subset=['air_out_idx', 'PRE_ER_ARREST_DT'], inplace = True)
patient.reset_index(drop = True, inplace = True)
patient

In [ ]:
len(hospital['air_out_idx'].unique())

In [ ]:
len(patient['air_out_idx'].unique())

In [ ]:
hospital.to_csv('data\\급성심장정지조사(08-19)SAS\\hospital.csv', encoding = 'euc-kr', index = False)
patient.to_csv('data\\급성심장정지조사(08-19)SAS\\patient.csv', encoding = 'euc-kr', index = False)